In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import random
from math import sqrt
print("packages instaled")

packages instaled


In [2]:
import os

path = r'C:\\Users\\Srikar\\Desktop\\the-movies-dataset'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.csv' in file:
            files.append(os.path.join(r, file))
for f in files:
    print(f)

C:\\Users\\Srikar\\Desktop\\the-movies-dataset\credits.csv
C:\\Users\\Srikar\\Desktop\\the-movies-dataset\keywords.csv
C:\\Users\\Srikar\\Desktop\\the-movies-dataset\links.csv
C:\\Users\\Srikar\\Desktop\\the-movies-dataset\links_small.csv
C:\\Users\\Srikar\\Desktop\\the-movies-dataset\movies_metadata.csv
C:\\Users\\Srikar\\Desktop\\the-movies-dataset\ratings.csv
C:\\Users\\Srikar\\Desktop\\the-movies-dataset\ratings_small.csv


In [3]:
'''print(os.path.getsize('C:\\Users\\Srikar\\Desktop\\the-movies-dataset\ratings.csv'))
print(os.stat('C:\\Users\\Srikar\\Desktop\\the-movies-dataset\ratings_small.csv').st_size)'''

"print(os.path.getsize('C:\\Users\\Srikar\\Desktop\\the-movies-dataset\ratings.csv'))\nprint(os.stat('C:\\Users\\Srikar\\Desktop\\the-movies-dataset\ratings_small.csv').st_size)"

## Data Collection/ Preprocessing

We will first read the movies data set and create a movies dataframe

In [4]:
movies = pd.read_csv(r'C:\\Users\\Srikar\\Desktop\\the-movies-dataset\movies_metadata.csv')
keep = ['id','original_title','release_date']
for col in movies.columns:
    if col not in keep:
        movies.drop(col, 1, inplace = True)

movies.head()

c:\users\srikar\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,original_title,release_date
0,862,Toy Story,1995-10-30
1,8844,Jumanji,1995-12-15
2,15602,Grumpier Old Men,1995-12-22
3,31357,Waiting to Exhale,1995-12-22
4,11862,Father of the Bride Part II,1995-02-10


In [5]:
movies = movies.rename(columns = {'id': 'movieID','original_title' : 'Title', 'release_date': 'Year'})
movies['Year'] = movies['Year'].str.extract('(\d\d\d\d)',expand=False)
movies.head()

,movieID,Title,Year
0,862,Toy Story,1995
1,8844,Jumanji,1995
2,15602,Grumpier Old Men,1995
3,31357,Waiting to Exhale,1995
4,11862,Father of the Bride Part II,1995


Some of the entries in the movieID column contain dahses. Here we are removing those entries

In [6]:
boolID = movies['movieID'].str.contains('-', regex = False)
boolID = ~boolID
movies['movieID'] = movies[boolID.values]['movieID']
movies.head()

,movieID,Title,Year
0,862,Toy Story,1995
1,8844,Jumanji,1995
2,15602,Grumpier Old Men,1995
3,31357,Waiting to Exhale,1995
4,11862,Father of the Bride Part II,1995


In [7]:
print(movies['movieID'].isna().sum())
#Because total NaN was only 3, number is insignificant enough to drop all NaN
movies = movies.dropna(subset=['movieID'])
movies.head()
print(movies['movieID'].isna().sum())

3
0


In [8]:
movies['movieID'] = movies['movieID'].apply(pd.to_numeric)
movies.dtypes

movieID     int64
Title      object
Year       object
dtype: object

Now, lets read multiple user ratings data. This will be our ratings dataframe.

In [9]:
ratings = pd.read_csv(r'C:\\Users\\Srikar\\Desktop\\the-movies-dataset\ratings_small.csv', engine = 'python')
ratings = ratings.rename(columns = {'userId':'userID','movieId':'movieID','rating':'Rating','timestamp':'Timestamp'})
ratings.head()

,userID,movieID,Rating,Timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


We will not create a input dataframe containing movies we have watched and their respective ratings. (All chosen at random)

In [38]:
titles = list(movies['Title'])
user_titles = random.sample(titles,5)
potential_ratings = [0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0]
user_ratings = random.sample(potential_ratings,5)
user_ratings
d = {'Title':user_titles,'Rating':user_ratings}
InputMovies = pd.DataFrame(d)
InputMovies.head()

,Title,Rating
0,Who Are the DeBolts? And Where Did They Get Ni...,5.0
1,The Woman in Question,4.5
2,L'ennui,3.0
3,Let's Get Harry,4.0
4,An American Romance,3.5


In [39]:
#Filtering out the movies df by titles in InputMovies
inputId = movies[movies['Title'].isin(InputMovies['Title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
InputMovies = pd.merge(inputId, InputMovies)
InputMovies = InputMovies.drop('Year', 1)
InputMovies.head()

,movieID,Title,Rating
0,27256,The Woman in Question,4.5
1,55107,L'ennui,3.0
2,32031,Let's Get Harry,4.0
3,114514,Who Are the DeBolts? And Where Did They Get Ni...,5.0
4,118059,An American Romance,3.5


In [40]:
#Filtering out users that have watched the same movies as the input has watched and storing it
userSubset = ratings[ratings['movieID'].isin(InputMovies['movieID'].tolist())]
userSubset.head()

,userID,movieID,Rating,Timestamp
7569,48,32031,3.0,1339446959
12666,79,32031,3.5,1182556308
28000,205,32031,3.0,1442152434
29783,213,32031,3.5,1462638838
34953,250,32031,4.0,1469808194


In [41]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userID'])

In [42]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [43]:
userSubsetGroup = userSubsetGroup[0:100]

## Linear Algebra

Here we are going to calculate the similatiry of other users to our input users. I have chosen to do this by Pearson Correlation

In [44]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    print(name, group)
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieID')
    InputMovies = InputMovies.sort_values(by='movieID')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = InputMovies[InputMovies['movieID'].isin(group['movieID'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['Rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['Rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


48       userID  movieID  Rating   Timestamp
7569      48    32031     3.0  1339446959
79        userID  movieID  Rating   Timestamp
12666      79    32031     3.5  1182556308
205        userID  movieID  Rating   Timestamp
28000     205    32031     3.0  1442152434
213        userID  movieID  Rating   Timestamp
29783     213    32031     3.5  1462638838
250        userID  movieID  Rating   Timestamp
34953     250    32031     4.0  1469808194
283        userID  movieID  Rating   Timestamp
38747     283    32031     3.5  1131817120
287        userID  movieID  Rating   Timestamp
39423     287    32031     4.0  1469163172
328        userID  movieID  Rating   Timestamp
45888     328    32031     3.5  1148000823
384        userID  movieID  Rating   Timestamp
53048     384    32031     3.0  1154367822
468        userID  movieID  Rating   Timestamp
66729     468    32031     2.5  1296203064
501        userID  movieID  Rating   Timestamp
71498     501    32031     2.5  1309493175
580        use

PearsonDF can also be known as the similarity matrix

In [45]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userID'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userID
0,0,48
1,0,79
2,0,205
3,0,213
4,0,250


In [46]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userID
0,0,48
1,0,79
2,0,205
3,0,213
4,0,250


In [47]:
topUsersRating=topUsers.merge(ratings, left_on='userID', right_on='userID', how='inner')
topUsersRating.head()

,similarityIndex,userID,movieID,Rating,Timestamp
0,0,48,1,4.0,1318796720
1,0,48,2,3.5,1322169967
2,0,48,34,3.0,1322169717
3,0,48,110,4.0,1319746142
4,0,48,111,3.0,1439687078


In [48]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Rating']
topUsersRating.head()

,similarityIndex,userID,movieID,Rating,Timestamp,weightedRating
0,0,48,1,4.0,1318796720,0.0
1,0,48,2,3.5,1322169967,0.0
2,0,48,34,3.0,1322169717,0.0
3,0,48,110,4.0,1319746142,0.0
4,0,48,111,3.0,1439687078,0.0


In [49]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieID').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieID,,
1,0,0.0
2,0,0.0
3,0,0.0
5,0,0.0
6,0,0.0


In [50]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieID'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieID
movieID,,
1,NaN,1
2,NaN,2
3,NaN,3
5,NaN,5
6,NaN,6


In [51]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieID
movieID,,
1,NaN,1
2,NaN,2
3,NaN,3
5,NaN,5
6,NaN,6
7,NaN,7
10,NaN,10
11,NaN,11
12,NaN,12


In [52]:
movies.loc[movies['movieID'].isin(recommendation_df.head(10)['movieID'].tolist())]

,movieID,Title,Year
17,5,Four Rooms,1995
256,11,Star Wars,1977
351,13,Forrest Gump,1994
474,6,Judgment Night,1993
4342,2,Ariel,1988
6232,12,Finding Nemo,2003
12947,3,Varjoja paratiisissa,1986


This will be the reccomended movies based on our ratings